In [15]:
import pandas as pd
import numpy as np
import keras
from datetime import datetime
from keras.models import Sequential 
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from keras import losses
from keras.constraints import maxnorm
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import pickle
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from commons import mean_absolute_percentage_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score
from keras.models import load_model
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Audio
#sound_file = 'beep.wav'
from keras.callbacks import *

### Bitcoin

In [17]:
regs = pd.read_csv("btc_reg.csv")
btc = pd.read_csv("btc_Data.csv")
for i in range(len(btc['Date'])):
    btc['Date'][i]  =  datetime.strptime(btc['Date'][i], '%Y/%m/%d')

btc = btc.set_index("Date")

/Users/liuxuyang/opt/anaconda3/envs/mytensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [18]:
btcData = btc[regs.columns]

In [20]:
btcData['returns'] = btcData['priceUSD'].pct_change()

/Users/liuxuyang/opt/anaconda3/envs/mytensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
Data = btcData.drop(columns=['priceUSD'])
Data = Data[1:]

In [28]:
Data 

,difficulty14mom,difficulty30mom,difficulty3rsi,difficulty90mom,difficulty90sma,hashrate90std,hashrate90var,median_transaction_fee90rocUSD,median_transaction_feeUSD,mining_profitability,...,price90wmaUSD,sentinusd90emaUSD,sentinusdUSD,top100cap,top100cap14trx,top100cap30rsi,top100cap30trx,top100cap90rsi,top100cap90trx,returns
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-18,1.621380e+02,1.641620e+02,100.00,1.737230e+02,2.055500e+01,8.384389e+08,1.757449e+17,0.000,0.000,401834.0000,...,0.0,0,2.612000e+03,25.782,0.139,71.191,0.308,79.756,0.550,0.466667
2010-07-19,1.621380e+02,1.641620e+02,100.00,1.737230e+02,2.248600e+01,8.819952e+08,1.944789e+17,0.000,0.000,481473.0000,...,0.0,0,4.047000e+03,25.685,0.123,68.863,0.300,78.999,0.547,0.183196
2010-07-20,1.584060e+02,1.641620e+02,100.00,1.735570e+02,2.441400e+01,9.280542e+08,2.153212e+17,0.000,0.000,431831.0000,...,0.0,0,2.341000e+03,25.602,0.107,66.923,0.292,78.355,0.544,-0.088475
2010-07-21,1.580420e+02,1.641620e+02,100.00,1.700790e+02,2.630400e+01,9.761949e+08,2.382391e+17,0.000,0.000,460783.0000,...,0.0,0,2.122000e+03,25.638,0.092,67.334,0.284,78.431,0.541,-0.020434
2010-07-22,1.580420e+02,1.641620e+02,100.00,1.700790e+02,2.819400e+01,1.019474e+09,2.598318e+17,0.000,0.000,334641.0000,...,0.0,0,2.570000e+03,25.604,0.078,66.526,0.275,78.166,0.537,-0.153846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-24,1.882952e+11,1.148115e+11,99.86,8.598887e+12,3.384949e+13,4.612156e+19,5.317997e+38,165.380,0.818,0.0577,...,18820.0,28544682257,6.114577e+09,15.236,-0.080,40.118,-0.042,45.140,0.001,0.008075
2022-11-25,1.882952e+11,1.148115e+11,99.86,8.598887e+12,3.394503e+13,4.542600e+19,5.158804e+38,214.464,0.812,0.0541,...,18759.0,28076113671,7.224812e+09,15.231,-0.075,39.880,-0.043,45.047,0.001,-0.006565
2022-11-26,1.882952e+11,1.148115e+11,99.86,8.598887e+12,3.404058e+13,4.624135e+19,5.345655e+38,255.004,0.809,0.0571,...,18701.0,27566714445,4.898449e+09,15.233,-0.070,40.069,-0.043,45.106,0.001,0.004001


In [163]:
# divide X and Y
X = Data.iloc[:,0:]
#Y = Data['returns']   # 用returns的话就用这一行，然后把下一行comment掉
Y = btcData['priceUSD'][1:] # 反之亦然

In [164]:
X.head(3)

,difficulty14mom,difficulty30mom,difficulty3rsi,difficulty90mom,difficulty90sma,hashrate90std,hashrate90var,median_transaction_fee90rocUSD,median_transaction_feeUSD,mining_profitability,...,price90wmaUSD,sentinusd90emaUSD,sentinusdUSD,top100cap,top100cap14trx,top100cap30rsi,top100cap30trx,top100cap90rsi,top100cap90trx,returns
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-18,162.138,164.162,100.0,173.723,20.555,838438881.0,1.757449e+17,0.0,0.0,401834.0,...,0.0,0,2612.0,25.782,0.139,71.191,0.308,79.756,0.550,0.466667
2010-07-19,162.138,164.162,100.0,173.723,22.486,881995244.0,1.944789e+17,0.0,0.0,481473.0,...,0.0,0,4047.0,25.685,0.123,68.863,0.300,78.999,0.547,0.183196
2010-07-20,158.406,164.162,100.0,173.557,24.414,928054231.0,2.153212e+17,0.0,0.0,431831.0,...,0.0,0,2341.0,25.602,0.107,66.923,0.292,78.355,0.544,-0.088475


In [165]:
# Split into three data sets
X_train = X['2016-01-01':'2019-12-31']
X_val = X['2020-01-01':'2021-05-31']
X_test = X['2021-06-01':'2023-01-01']

Y_train = Y['2016-01-01':'2019-12-31']
Y_val = Y['2020-01-01':'2021-05-31']
Y_test = Y['2021-06-01':'2023-01-01']

In [166]:
estimators=[]
estimators.append(['mixmax',MinMaxScaler()])
estimators.append(['robust',RobustScaler()])
scale=Pipeline(estimators,verbose=True)
scale.fit(X_train)

[Pipeline] ............ (step 1 of 2) Processing mixmax, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing robust, total=   0.0s


Pipeline(steps=[('mixmax', MinMaxScaler()), ['robust', RobustScaler()]],
         verbose=True)

In [167]:
X_train=scale.transform(X_train)
X_test=scale.transform(X_test)
X_val = scale.transform(X_val)

In [168]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [169]:
from keras import regularizers
# define neural network model
shape=X.shape[1]
def sequential_model(initializer='normal', activation='relu', neurons=300, NUM_FEATURES=shape):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_shape=(NUM_FEATURES,), kernel_initializer=initializer, activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(1, activation=activation, kernel_initializer=initializer))
    # Compile model
    adam=keras.optimizers.adam(lr=lr_schedule(0), amsgrad=True)
    #sgd=keras.optimizers.SGD(learning_rate=0.08, momentum=0.9, nesterov=False)
    model.compile(loss='logcosh', optimizer=adam, metrics=['mae'])
    return model

In [170]:
#mcp_save = ModelCheckpoint('trained_models/ANN_reg_seven_new.hdf5', save_best_only=True, monitor='val_loss', mode='auto')
#earlyStopping = EarlyStopping(monitor='val_loss', patience=100,verbose=1, mode='auto')

In [175]:
regressor=KerasRegressor(build_fn=sequential_model,epochs=1000,verbose=1, shuffle=True,use_multiprocessing=True)

In [176]:
regressor.fit(X_train,Y_train,validation_data=(X_val, Y_val))
#Audio(sound_file,autoplay=True)

Learning rate:  0.001
Train on 1461 samples, validate on 517 samples
Epoch 1/1000
1461/1461 [==============================] - 1s 713us/step - loss: 4542.1763 - mae: 4542.8691 - val_loss: 17803.1971 - val_mae: 17803.8906
Epoch 2/1000
1461/1461 [==============================] - 0s 275us/step - loss: 2609.4767 - mae: 2610.1697 - val_loss: 3853.4012 - val_mae: 3854.0945
Epoch 3/1000
1461/1461 [==============================] - 0s 274us/step - loss: 1046.6732 - mae: 1047.3660 - val_loss: 3140.4699 - val_mae: 3141.1631
Epoch 4/1000
1461/1461 [==============================] - 0s 278us/step - loss: 793.9743 - mae: 794.6665 - val_loss: 2453.1101 - val_mae: 2453.8032
Epoch 5/1000
1461/1461 [==============================] - 0s 278us/step - loss: 652.6379 - mae: 653.3306 - val_loss: 2329.8031 - val_mae: 2330.4961
Epoch 6/1000
1461/1461 [==============================] - 0s 278us/step - loss: 593.3032 - mae: 593.9957 - val_loss: 1730.6173 - val_mae: 1731.3104
Epoch 7/1000
1461/1461 [===========

1461/1461 [==============================] - 0s 291us/step - loss: 96.3191 - mae: 97.0082 - val_loss: 1233.9854 - val_mae: 1234.6786
Epoch 56/1000
1461/1461 [==============================] - 0s 309us/step - loss: 122.0347 - mae: 122.7245 - val_loss: 1419.1776 - val_mae: 1419.8706
Epoch 57/1000
1461/1461 [==============================] - 0s 311us/step - loss: 139.7451 - mae: 140.4330 - val_loss: 1433.4956 - val_mae: 1434.1888
Epoch 58/1000
1461/1461 [==============================] - 0s 294us/step - loss: 101.1396 - mae: 101.8274 - val_loss: 1596.5171 - val_mae: 1597.2103
Epoch 59/1000
1461/1461 [==============================] - 0s 290us/step - loss: 106.4784 - mae: 107.1676 - val_loss: 1241.3191 - val_mae: 1242.0123
Epoch 60/1000
1461/1461 [==============================] - 0s 306us/step - loss: 86.5931 - mae: 87.2815 - val_loss: 1160.7310 - val_mae: 1161.4224
Epoch 61/1000
1461/1461 [==============================] - 0s 305us/step - loss: 99.9678 - mae: 100.6578 - val_loss: 1660.64

1461/1461 [==============================] - 0s 287us/step - loss: 75.1854 - mae: 75.8693 - val_loss: 1604.3223 - val_mae: 1605.0154
Epoch 111/1000
1461/1461 [==============================] - 0s 267us/step - loss: 80.0044 - mae: 80.6902 - val_loss: 1251.9697 - val_mae: 1252.6630
Epoch 112/1000
1461/1461 [==============================] - 0s 274us/step - loss: 68.5362 - mae: 69.2203 - val_loss: 1602.2688 - val_mae: 1602.9620
Epoch 113/1000
1461/1461 [==============================] - 0s 275us/step - loss: 77.5554 - mae: 78.2377 - val_loss: 1421.2114 - val_mae: 1421.9037
Epoch 114/1000
1461/1461 [==============================] - 0s 269us/step - loss: 79.9100 - mae: 80.5963 - val_loss: 1172.7373 - val_mae: 1173.4304
Epoch 115/1000
1461/1461 [==============================] - 0s 277us/step - loss: 86.8860 - mae: 87.5692 - val_loss: 1373.6040 - val_mae: 1374.2971
Epoch 116/1000
1461/1461 [==============================] - 0s 275us/step - loss: 88.6030 - mae: 89.2913 - val_loss: 1205.7977 

1461/1461 [==============================] - 0s 271us/step - loss: 77.4210 - mae: 78.1065 - val_loss: 1724.7867 - val_mae: 1725.4799
Epoch 166/1000
1461/1461 [==============================] - 0s 265us/step - loss: 73.3359 - mae: 74.0207 - val_loss: 1831.4773 - val_mae: 1832.1704
Epoch 167/1000
1461/1461 [==============================] - 0s 263us/step - loss: 66.5478 - mae: 67.2291 - val_loss: 1447.9736 - val_mae: 1448.6669
Epoch 168/1000
1461/1461 [==============================] - 0s 272us/step - loss: 77.1604 - mae: 77.8462 - val_loss: 1342.2488 - val_mae: 1342.9420
Epoch 169/1000
1461/1461 [==============================] - 0s 267us/step - loss: 61.4836 - mae: 62.1692 - val_loss: 1577.0740 - val_mae: 1577.7673
Epoch 170/1000
1461/1461 [==============================] - 0s 273us/step - loss: 69.6202 - mae: 70.3018 - val_loss: 703.8638 - val_mae: 704.5567
Epoch 171/1000
1461/1461 [==============================] - 0s 268us/step - loss: 100.3975 - mae: 101.0851 - val_loss: 1525.2374 

1461/1461 [==============================] - 0s 341us/step - loss: 76.2374 - mae: 76.9217 - val_loss: 1370.6034 - val_mae: 1371.2965
Epoch 221/1000
1461/1461 [==============================] - 0s 274us/step - loss: 73.1830 - mae: 73.8669 - val_loss: 1132.1940 - val_mae: 1132.8851
Epoch 222/1000
1461/1461 [==============================] - 0s 271us/step - loss: 54.4735 - mae: 55.1539 - val_loss: 1568.6205 - val_mae: 1569.3136
Epoch 223/1000
1461/1461 [==============================] - 0s 268us/step - loss: 49.5219 - mae: 50.2060 - val_loss: 1479.3499 - val_mae: 1480.0432
Epoch 224/1000
1461/1461 [==============================] - 0s 270us/step - loss: 48.6452 - mae: 49.3252 - val_loss: 1199.4439 - val_mae: 1200.1371
Epoch 225/1000
1461/1461 [==============================] - 0s 306us/step - loss: 66.5411 - mae: 67.2269 - val_loss: 1597.2397 - val_mae: 1597.9329
Epoch 226/1000
1461/1461 [==============================] - 0s 281us/step - loss: 53.2838 - mae: 53.9661 - val_loss: 1424.8398 

...

1461/1461 [==============================] - 0s 280us/step - loss: 38.1682 - mae: 38.8488 - val_loss: 1452.1356 - val_mae: 1452.8289
Epoch 771/1000
1461/1461 [==============================] - 0s 270us/step - loss: 35.5897 - mae: 36.2645 - val_loss: 1399.0100 - val_mae: 1399.7034
Epoch 772/1000
1461/1461 [==============================] - 0s 280us/step - loss: 42.9380 - mae: 43.6168 - val_loss: 1318.1008 - val_mae: 1318.7938
Epoch 773/1000
1461/1461 [==============================] - 0s 272us/step - loss: 43.2023 - mae: 43.8813 - val_loss: 1325.8935 - val_mae: 1326.5858
Epoch 774/1000
1461/1461 [==============================] - 0s 269us/step - loss: 44.1093 - mae: 44.7899 - val_loss: 1284.6074 - val_mae: 1285.2997
Epoch 775/1000
1461/1461 [==============================] - 0s 266us/step - loss: 45.4662 - mae: 46.1508 - val_loss: 1215.9768 - val_mae: 1216.6699
Epoch 776/1000
1461/1461 [==============================] - 0s 296us/step - loss: 39.0265 - mae: 39.7043 - val_loss: 1506.0745 

1461/1461 [==============================] - 0s 268us/step - loss: 51.2658 - mae: 51.9484 - val_loss: 1402.5957 - val_mae: 1403.2889
Epoch 826/1000
1461/1461 [==============================] - 0s 271us/step - loss: 42.5473 - mae: 43.2225 - val_loss: 1444.6925 - val_mae: 1445.3857
Epoch 827/1000
1461/1461 [==============================] - 0s 273us/step - loss: 41.4212 - mae: 42.1001 - val_loss: 1059.4295 - val_mae: 1060.1227
Epoch 828/1000
1461/1461 [==============================] - 0s 266us/step - loss: 53.6947 - mae: 54.3761 - val_loss: 1341.1427 - val_mae: 1341.8359
Epoch 829/1000
1461/1461 [==============================] - 0s 269us/step - loss: 40.0758 - mae: 40.7550 - val_loss: 1174.7327 - val_mae: 1175.4257
Epoch 830/1000
1461/1461 [==============================] - 0s 269us/step - loss: 51.9253 - mae: 52.6090 - val_loss: 1554.0536 - val_mae: 1554.7462
Epoch 831/1000
1461/1461 [==============================] - 0s 270us/step - loss: 37.8079 - mae: 38.4837 - val_loss: 1231.4851 

1461/1461 [==============================] - 0s 302us/step - loss: 40.0376 - mae: 40.7168 - val_loss: 1733.7028 - val_mae: 1734.3961
Epoch 881/1000
1461/1461 [==============================] - 0s 282us/step - loss: 38.7772 - mae: 39.4531 - val_loss: 1578.8400 - val_mae: 1579.5331
Epoch 882/1000
1461/1461 [==============================] - 0s 301us/step - loss: 33.8257 - mae: 34.5053 - val_loss: 1379.5220 - val_mae: 1380.2152
Epoch 883/1000
1461/1461 [==============================] - 0s 312us/step - loss: 41.8069 - mae: 42.4850 - val_loss: 1157.3325 - val_mae: 1158.0253
Epoch 884/1000
1461/1461 [==============================] - 0s 306us/step - loss: 42.6225 - mae: 43.3022 - val_loss: 1418.0006 - val_mae: 1418.6937
Epoch 885/1000
1461/1461 [==============================] - 0s 294us/step - loss: 37.3401 - mae: 38.0201 - val_loss: 1413.6460 - val_mae: 1414.3392
Epoch 886/1000
1461/1461 [==============================] - 0s 302us/step - loss: 51.2495 - mae: 51.9293 - val_loss: 1327.8432 

1461/1461 [==============================] - 0s 269us/step - loss: 41.5086 - mae: 42.1892 - val_loss: 1370.0331 - val_mae: 1370.7260
Epoch 936/1000
1461/1461 [==============================] - 0s 269us/step - loss: 32.0226 - mae: 32.6992 - val_loss: 1337.6936 - val_mae: 1338.3867
Epoch 937/1000
1461/1461 [==============================] - 0s 268us/step - loss: 40.7207 - mae: 41.3981 - val_loss: 1374.6977 - val_mae: 1375.3904
Epoch 938/1000
1461/1461 [==============================] - 0s 271us/step - loss: 36.2858 - mae: 36.9642 - val_loss: 1071.5209 - val_mae: 1072.2140
Epoch 939/1000
1461/1461 [==============================] - 0s 270us/step - loss: 40.2708 - mae: 40.9471 - val_loss: 1214.2865 - val_mae: 1214.9797
Epoch 940/1000
1461/1461 [==============================] - 0s 271us/step - loss: 49.1143 - mae: 49.7996 - val_loss: 1443.0293 - val_mae: 1443.7224
Epoch 941/1000
1461/1461 [==============================] - 0s 282us/step - loss: 45.6330 - mae: 46.3120 - val_loss: 1185.7036 

1461/1461 [==============================] - 0s 287us/step - loss: 37.8379 - mae: 38.5148 - val_loss: 1351.9616 - val_mae: 1352.6549
Epoch 991/1000
1461/1461 [==============================] - 0s 309us/step - loss: 34.2751 - mae: 34.9499 - val_loss: 1449.5983 - val_mae: 1450.2916
Epoch 992/1000
1461/1461 [==============================] - 0s 318us/step - loss: 36.8636 - mae: 37.5439 - val_loss: 1296.3126 - val_mae: 1297.0059
Epoch 993/1000
1461/1461 [==============================] - 0s 316us/step - loss: 39.1588 - mae: 39.8350 - val_loss: 1542.4102 - val_mae: 1543.1034
Epoch 994/1000
1461/1461 [==============================] - 0s 309us/step - loss: 32.8260 - mae: 33.5020 - val_loss: 1253.7422 - val_mae: 1254.4354
Epoch 995/1000
1461/1461 [==============================] - 0s 321us/step - loss: 44.6902 - mae: 45.3684 - val_loss: 1322.0887 - val_mae: 1322.7817
Epoch 996/1000
1461/1461 [==============================] - 0s 312us/step - loss: 45.5949 - mae: 46.2767 - val_loss: 1523.4362 

In [177]:
# for check
y_train_pred=regressor.predict(X_train)
r2_score(Y_train, y_train_pred) #training score

1461/1461 [==============================] - 0s 150us/step


0.9997976525406873

In [178]:
y_pred=regressor.predict(X_test)
pre_df = pd.DataFrame(zip(Y_test,y_pred),columns=['y_test','y_pred'])
pre_df.index = Y_test.index
pre_df

546/546 [==============================] - 0s 110us/step


,y_test,y_pred
Date,,
2021-06-01,36599.0,39957.894531
2021-06-02,37295.0,40503.027344
2021-06-03,38539.0,41449.289062
2021-06-04,37117.0,40811.390625
2021-06-05,36607.0,40478.691406
...,...,...
2022-11-24,16604.0,20915.765625
2022-11-25,16495.0,20869.060547
2022-11-26,16561.0,21003.966797


In [179]:
r2=r2_score(Y_test,y_pred) #testing score/ r^2
r2

0.8989174061600431

In [180]:
rmse=np.sqrt(mean_squared_error(Y_test,y_pred)) #rmse
rmse#太特么大了，感觉数据集划分有问题

4185.654414668638

In [181]:
pre_df['pred_returns'] = pre_df['y_pred'].pct_change()

In [182]:
pre_df

,y_test,y_pred,pred_returns
Date,,,
2021-06-01,36599.0,39957.894531,NaN
2021-06-02,37295.0,40503.027344,0.013643
2021-06-03,38539.0,41449.289062,0.023363
2021-06-04,37117.0,40811.390625,-0.015390
2021-06-05,36607.0,40478.691406,-0.008152
...,...,...,...
2022-11-24,16604.0,20915.765625,0.004465
2022-11-25,16495.0,20869.060547,-0.002233
2022-11-26,16561.0,21003.966797,0.006464
